# Introduction
The goal here is to visually inspect the hidden features computed by the model at each intermediate layer.

### Imports
Import libraries and write settings here.

## General packages

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model, Model, Sequential
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers import Input, Dropout, Activation, Conv2D, MaxPooling2D, UpSampling2D, Lambda, BatchNormalization
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import callbacks, initializers, layers, models, optimizers
from keras import backend as K

from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt

## Custom modules 

In [ ]:
import sys
%load_ext autoreload

%autoreload 2

repo_path = Path("..")

sys.path.append("..")
import visualization_utils

# Visualisation


In [ ]:
# set paths where to read images and masks
# SAMPLE_IMG_PATH = repo_path / "DATASET/test_tr_opt/sample_valid/images"
# SAMPLE_MASKS_PATH = repo_path / "DATASET/test_tr_opt/sample_valid/masks"
# SAMPLE_IMG_PATH = repo_path / "DATASET/test/all_images/images"
# SAMPLE_MASKS_PATH = repo_path / "DATASET/test/all_masks/masks"
SAMPLE_IMG_PATH = Path(
    '/home/luca/PycharmProjects/cell-segmentation/dataset/yellow/sample/images')
SAMPLE_MASKS_PATH = Path(
    '/home/luca/PycharmProjects/cell-segmentation/dataset/yellow/sample/masks')

# MODEL
model_name = "c-ResUnet.h5"
# model_name = "c-ResUnet_noWM.h5"
model_path = "{}/model_results/{}".format(repo_path, model_name)

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.2, 0.8, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

# dice loss


def dice_coef(y_true, y_pred):
    """Generate the 'Dice' coefficient for the provided prediction.
    Args:
        y_true: The expected/desired output mask.
        y_pred: The actual/predicted mask.
    Returns:
        The Dice coefficient between the expected and actual outputs. Values
        closer to 1 are considered 'better'.
    """
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    """Model loss function using the 'Dice' coefficient.
    Args:
        y_true: The expected/desired output mask.
        y_pred: The actual/predicted mask.
    Returns:
        The corresponding loss, related to the dice coefficient between the expected
        and actual outputs. Values closer to 0 are considered 'better'.
    """
    return -dice_coef(y_true, y_pred)


def create_weighted_binary_crossentropy(zero_weight, one_weight):

    def weighted_binary_crossentropy(y_true, y_pred):

        b_ce = K.binary_crossentropy(y_true, y_pred)

        # Apply the weights
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce

        # Return the mean error
        return K.mean(weighted_b_ce)

    return weighted_binary_crossentropy

## Load model

In [ ]:
WeightedLoss = create_weighted_binary_crossentropy(1, 1.5)

model = load_model(model_path, custom_objects={'mean_iou': mean_iou, 'dice_coef': dice_coef,
                                               'weighted_binary_crossentropy': WeightedLoss}, compile=False)

In [ ]:
model.summary()

## Check feature maps 

In [ ]:
def label_func(p):
    return Path(str(p).replace('images', 'masks'))


def get_input(img_path, _div=255.):

    img_rgb = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    mask_path = label_func(img_path)
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)

    # predictions
    img_rgb = np.expand_dims(img_rgb, 0)
    inp = img_rgb/_div
    return inp, mask


def get_activations(img, model, layer):
    feature_extractor = Model(inputs=model.inputs, outputs=layer.output)
    activations = feature_extractor.predict(img)
    return activations


def plot_featuremap(img_path, layers, debug=False):
    inp, mask = get_input(img_path)
    # original + true objects
    plt.imshow(np.squeeze(inp),
               cmap=plt.cm.RdBu, aspect="auto")
    plt.contour(mask, [0.5], linewidths=1.2, colors='w')
    plt.title(img_path.name)
    plt.show()

    fig_shape_dict = {1: (1, 1, (8, 8)),
                      16: (4, 4, (12, 12)),
                      32: (4, 8, (16, 16)),
                      64: (8, 8, (20, 20)),
                      128: (16, 8, (24, 24))
                      }

    for l in layers:
        activations = get_activations(inp, model, l)

        n_rows, n_cols, figsize = fig_shape_dict[activations.shape[3]]
        row_spacing, col_spacing = 0.02, 0

        fig = plt.figure(figsize=figsize)
        fig.suptitle(
            f"{l.name}, feature size: {np.squeeze(activations[:, :, :, 0]).shape}")
        gs = plt.GridSpec(n_rows, n_cols,
                          wspace=0, hspace=0,
                          #                           top=0.95, bottom=0.05,
                          height_ratios=[1-col_spacing*n_rows]*n_rows,
                          left=0.01, right=0.9,
                          width_ratios=[1-row_spacing*n_cols]*n_cols,
                          figure=fig,
                          )

        for i in range(n_rows):
            for j in range(n_cols):
                ax = plt.subplot(gs[i, j])
                feat_map = np.squeeze(activations[:, :, :, n_cols * i + j])
                if debug: 
                    print('Layer name:', l.name, 'feature n.', n_cols * i + j)
                    print(feat_map)
                im = ax.imshow(feat_map, cmap='jet')
                ax.axis('off')

        fig.subplots_adjust(right=0.9)  # , left=0.05, top=0.95, bottom=0.05)
        cbar_ax = fig.add_axes([0.91, 0.05, 0.03, 0.9])
        # , ticks=[0, 0.2, 0.4, 0.6, 0.8, 0.99])
        cbar = fig.colorbar(im, cax=cbar_ax)
        cbar.ax.tick_params(labelsize=12)
    # cbar.ax.set_yticklabels(['0', '0.2', '0.4', '0.6', '0.8', '1'])
    plt.show()

In [ ]:
p = [*Path(SAMPLE_IMG_PATH).iterdir()][0]


LAYERS = model.layers[5:6]
for layer in LAYERS:
    if 'input' in layer.name:
        continue
    print(layer.name)
    plot_featuremap(p, [layer])